### KFServing - Financial Data

In [67]:
# Create namespace for kfserving
!kubectl create ns kfserving

namespace/kfserving created


In [68]:
# Enable Inference Service
!kubectl label namespace kfserving serving.kubeflow.org/inferenceservice=enabled

namespace/kfserving labeled


In [69]:
#Apply the kferving CRD
!kubectl apply -f kfserving.yaml -n kfserving

inferenceservice.serving.kubeflow.org/churn created


In [81]:
#check the status of the pods
!kubectl get pods -n kfserving

NAME                                                        READY   STATUS    RESTARTS   AGE
churn-predictor-default-ckn58-deployment-5759559657-brc4l   2/2     Running   0          15s


In [82]:
# Check the status of the Inference service
!kubectl get inferenceservice -n kfserving

NAME    URL                                  READY   DEFAULT TRAFFIC   CANARY TRAFFIC   AGE
churn   http://churn.kfserving.example.com   True    100                                4m12s


In [12]:
#Check the container logs - very useful for debugging
!kubectl logs -c storage-initializer churn-predictor-default-pbs66-deployment-58cf555fb5-64vlz -n kfserving

[I 210216 10:35:33 initializer-entrypoint:13] Initializing, args: src_uri [gs://mavencode-churn-model/customer-churn/churn01] dest_path[ [/mnt/models]
[I 210216 10:35:33 storage:35] Copying contents of gs://mavencode-churn-model/customer-churn/churn01 to local
[W 210216 10:35:33 _default:246] Authentication failed using Compute Engine authentication due to unavailable metadata server.
[I 210216 10:35:33 storage:111] Downloading: /mnt/models/model.joblib
[I 210216 10:35:33 storage:60] Successfully copied gs://mavencode-churn-model/customer-churn/churn01 to /mnt/models


## Run these commands on the CLI

### These are environmental variabled required for the prediction request

In [22]:
export MODEL_NAME=churn

In [23]:
!echo $MODEL_NAME

churn


In [43]:
export INGRESS_HOST=$(kubectl get po -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].status.hostIP}')

In [15]:
export INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')

In [45]:
!echo $INGRESS_HOST

[172.16.0.108]


In [16]:
!echo $INGRESS_PORT

[31380]


In [49]:
export SERVICE_HOSTNAME=$(kubectl get inferenceservice -n kfserving churn -o jsonpath='{.status.url}' | cut -d "/" -f 3)

In [56]:
!echo $SERVICE_HOSTNAME

[churn.kfserving.example.com]


In [ ]:
#navigate to the right folder
cd KubeflowTraining/'Day 3'/'Kf serving'

In [72]:
export INPUT_PATH=@kfserving_input.json

In [70]:
!cat kfserving_input.json

{
    "instances": [
        [0.672, 32, 6, 0.0, 2, 1, 1, 0.895494, 1, 0, 0, 0, 1]
    ]
}

In [13]:
!curl -v -H "Host: ${SERVICE_HOSTNAME}"  http://$INGRESS_HOST:$INGRESS_PORT/v1/models/$MODEL_NAME:predict -d $INPUT_PATH

* Could not resolve host: 
* Closing connection 0
curl: (6) Could not resolve host: 
